方案一：基于多模态嵌入模型（Embedding-based Multi-modal RAG）
1. 原理
    使用 多模态嵌入模型（如 CLIP、SigLIP、OpenCLIP） 将图像与文本映射到同一向量空间
    所有数据统一用向量检索（Vector Search）
    检索出的图文块被送入多模态模型（如 GPT-4V / Gemini / QwenVL）进行最终答案生成

2. 优点
    成本最低：大部分推理发生在嵌入模型中，检索速度快
    架构简单：所有模态都向量化即可，不需要额外摘要
    易扩展：可加入更多模态（音频、视频帧等）

3. 缺点
    检索质量较差：多模态 embedding 的语义理解能力弱于大模型
    图像信息损失明显：embedding 模型对细节不敏感（文本、表格效果更差）
    难处理复杂图片（表格、公式、密集文本）

方案二：基于多模态大模型（图像摘要 + 文本摘要 + 多模态生成）
    速度最慢、成本最高、效果最好
1. 原理
    使用多模态大模型（如 GPT-4V）对图片进行 高质量图像摘要
    对文本通过大模型生成 结构化摘要 或精炼文本
    对“图像摘要 + 文本摘要”统一做嵌入 → 检索
    检索命中后，将 原始图像 + 原始文本 输入多模态大模型进行最终答案合成。

2. 优点
    检索效果最好：所有模态都由最强模型统一理解
    保真度最高：最终回答阶段重新输入原图，避免摘要损失信息
    适用任何复杂场景：表格、手写文字、流程图、PDF、截图等

3. 缺点
    成本最高：摘要阶段 + 生成阶段都使用大模型
    速度最慢：尤其数据量大时，离线摘要成本巨大
    维护复杂：摘要质量会影响检索稳定性

方案三：基于多模态大模型（图像摘要 + 文本向量检索 + 文本生成）
    成本适中、效果居中
    为什么说用了图片摘要却又没用？”
    因为方案三确实 只在预处理时使用图片摘要，不在生成时再读原图。
1. 原理
    使用多模态大模型（如 GPT-4V、Gemini、Qwen-VL）对图片生成 一次性高质量摘要
    图像摘要被当作纯文本处理
    所有检索都基于文本向量（如 text embedding）
    最终回答阶段只输入 文本块，不包含原图，不再做多模态生成

2. 优点
    成本中等：只在预处理用大模型，检索与生成使用便宜模型。
    效果比方案一好：因为文本摘要比 embedding 更捕捉语义。
    推理速度快：在线阶段全是文本处理。

3. 缺点
    无法重新理解原图：生成时不再读取图片 → 丢失视觉精度
    受摘要质量影响巨大：摘要写得不好 = 永久不准
    图像特征过度文本化：复杂图像内容可能被压缩或错译

方案对比
| 方案                     | 图片是否向量化 | 是否用 LLM 生成图片摘要 | 检索阶段    | 生成阶段是否重新看原图 | 成本 | 效果  |
| ----------------------- | ------------ | ------------------- | ---------- | ------------------ | --- | ---- |
| 方案一：多模态嵌入          | ✔️          | ❌                  | 向量检索     | 看原图              | 低  | 中偏低 |
| 方案二：多模态大模型全链路   | ❌           | ✔️                  | 摘要向量检索 | ✔️ 再看原图         |  高  | 最高  |
| 方案三：图片摘要 + 文本检索  | ❌          | ✔️                   | 文本向量检索 | ❌ 不再看原图        | 中  | 中   |


### 1.基于多模态向量模型的RAG

In [ ]:
# 1.1 初始化
import base64
import io
import os
import re
import shutil
import uuid
from typing import List

import dashscope
import markdown
from IPython.display import HTML, display
from PIL import Image
from langchain_chroma import Chroma
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from pydantic import BaseModel
from unstructured.documents.elements import Table, CompositeElement
from unstructured.partition.pdf import partition_pdf

# 环境配置
os.environ["TESSERACT_CMD"] = "/usr/bin/tesseract"

# 初始化
BASE_DIR = "/mnt/c/大模型/智泊大模型全栈教程总结/02-教材整理 L2/代码/Langchain/6.langchain高级RAG/data/"
RESOURCE_DIR = BASE_DIR + 'resources/'
TOOLS_DIR = BASE_DIR + 'tools/'
IMAGE_OUT_DIR = RESOURCE_DIR + 'images/'
PDF_PATH = RESOURCE_DIR + "978-7-5170-2271-8_1.pdf"
RESIZE_IMAGE_DIR = RESOURCE_DIR + "temp"

In [ ]:
# 1.2 基本函数定义
def show_plt_img(img_base64):
    """
    用于在 Jupyter Notebook 或类似环境中显示 Base64 编码的图像
    1. 使用 f-string 格式化创建一个 HTML 的 <img> 标签
    2. 标签的 src 属性使用 Data URL 格式:
    3. data:image/jpeg;base64, 表示这是一个 JPEG 图像的 Base64 编码数据
    4. 后面接上传入的 img_base64 字符串
    5. 使用 display(HTML(...)) 在 Notebook 中渲染这个 HTML 图像标签
    """
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    display(HTML(image_html))


def encode_image(img_path):
    """
    base64.b64encode(img_data) 将二进制数据编码为 Base64 字节串
    .decode('utf-8') 将 Base64 字节串转换为 UTF-8 字符串
    返回最终的 Base64 编码字符串
    """
    with open(img_path, "rb") as img_file:
        img_data = img_file.read()
        img_base64 = base64.b64encode(img_data).decode('utf-8')
        return img_base64


def display_answer(text: str):
    """
    输入参数:
    text: 字符串类型，包含文本内容和图像标记的特殊格式字符串
    标记格式:使用 <image> 和 </image> 作为图像路径的标记
    格式示例: "这是一些文本<image>path/to/image.jpg</image>更多文本"
    """
    start_tag = "<image>"
    end_tag = "</image>"

    # 根据<image>标签 分割文本：xxx<image>image</image>xxx => ['xxx','image</image>xxx']
    parts = text.split(start_tag)
    for part in parts:
        # 再根据</image>标签 分割文本：xxx => ['xxx'], image</image>xxx => ['image','xxx']
        chunks = part.split(end_tag)
        if len(chunks) > 1:
            # 存在图片
            image_path = chunks[0]
            context = chunks[1]
            img_base64 = encode_image(image_path)
            display(HTML(f'\n<img src="data:image/jpeg;base64,{img_base64}"/>\n'))
            display(HTML(markdown.markdown(context)))
        else:
            display(HTML(markdown.markdown(part)))


def resize_base64_image4tongyi(base64_string, max_size=(640, 480)):
    '''
    将Base64编码的图片进行缩放处理，并将缩放后的图片保存到本地文件系统，最后返回保存路径
    base64_string: Base64编码的图片字符串
        它是一个字符串，包含字母(A-Z, a-z)、数字(0-9)以及特殊字符(+/=)
        通常以类似"data:image/png;base64,"开头，后面跟着实际的Base64编码数据
        例如： "iVBORw0KGgoAAAANSUhEUgAAAAoAAAAKCAYAAACNMs+9AAAAFUlEQVR42mP8//+/JgMRQBOgLN4aAKkDZQ0V6XQZAAAAAElFTkSuQmCC"
    max_size: 一个元组，表示图片缩放后的最大尺寸，默认为(640, 480)
    '''
    # 解析图片
    img_data = base64.b64decode(base64_string)  # 将Base64字符串解码为二进制图片数据
    img = Image.open(io.BytesIO(img_data))  # 使用PIL库的Image.open()和io.BytesIO()从二进制数据创建图片对象

    width, height = img.size
    ratio = max(max_size[0] / width, max_size[1] / height)

    # 计算按比例缩放后的宽高
    new_width = int(width * ratio)
    new_height = int(height * ratio)

    # 缩放 Image.LANCZOS: 重采样滤波器
    resized_img = img.resize((new_width, new_height), Image.LANCZOS)

    # 保存图片到指定路径
    out_path = os.path.join(RESIZE_IMAGE_DIR, str(uuid.uuid4()) + ".jpg")
    resized_img.save(out_path)
    # 图片地址
    return out_path


def is_base64(s):
    """检查是否为base64数据"""
    # 检查是否为字符串
    if not isinstance(s, str):
        return False

    # 检查是否只包含 base64 允许的字符
    if not re.match(r'^[A-Za-z0-9+/]*={0,2}$', s):
        return False

    # 检查长度是否是 4 的倍数
    if len(s) % 4 != 0:
        return False

    # 尝试解码
    try:
        base64.b64decode(s, validate=True)
        return True
    except Exception:
        return False


def split_image_text_types(docs):
    """
    将文档和图片内容切分，分别存储到各自的列表中
    """
    images = []
    text = []
    for doc in docs:
        content = doc.page_content
        if is_base64(content):
            # 缩放图片
            resize_image = resize_base64_image4tongyi(content)
            images.append(resize_image)
        else:
            text.append(content)

    return {"images": images, "texts": text}

In [ ]:
# 1.3 解析文件
# 如果图片提取目录存在则删除重建
if os.path.exists(IMAGE_OUT_DIR):
    shutil.rmtree(IMAGE_OUT_DIR)
os.makedirs(IMAGE_OUT_DIR)

print('\n开始解析pdf文档' + '-' * 100)
pdf_data = partition_pdf(
    filename=PDF_PATH,
    extract_images_in_pdf=True,  # 提取图片
    infer_table_structure=True,  # 启用表格结构识别
    max_characters=4000,  # 每个文本块最大字符数
    new_after_n_chars=3800,  # 达到3800字符后分新块
    combine_text_under_n_chars=2000,  # 合并小于2000字符的文本块
    chunking_strategy="by_title",  # 按标题分块
    extract_image_block_output_dir=IMAGE_OUT_DIR,  # 图片提取路径
)

print("pdf_data 格式：[CompositeElement ，table，CompositeElement ，table,...]: ")
print(pdf_data)
print("查看 CompositeElement" + "-" * 100)
print(pdf_data[0].metadata.orig_elements)
print("查看 CompositeElement 的子节点" + "-" * 100)
print(pdf_data[0].metadata.orig_elements[1])

In [ ]:
# 1.4 提取表格与文本
tables = []
texts = []
for element in pdf_data:
    if isinstance(element, Table):
        tables.append(str(element))
    elif isinstance(element, CompositeElement):
        texts.append(str(element))
print(f"表格元素：{len(tables)}  文本元素：{len(texts)}")


In [ ]:
# 1.5 多模态嵌入模型
class MultiDashScopeEmbeddings(BaseModel, Embeddings):
    model: str = "multimodal-embedding-v1"

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        text_features = []
        for text in texts:
            resp = dashscope.MultiModalEmbedding.call(model=self.model, input=[{'text': text}])
            while resp.output is None:
                print(f"{text} 向量化失败！")
                resp = dashscope.MultiModalEmbedding.call(model=self.model, input=[{'text': text}])

            embeddings_list = resp.output['embeddings'][0]['embedding']
            text_features.append(embeddings_list)
        return text_features

    def embed_query(self, text: str) -> List[float]:
        resp = dashscope.MultiModalEmbedding.call(model=self.model, input=[{'text': text}])
        while resp.output is None:
            print(f"{text} 向量化失败！")
            resp = dashscope.MultiModalEmbedding.call(model=self.model, input=[{'text': text}])

        embeddings_list = resp.output['embeddings'][0]['embedding']
        return embeddings_list

    def embed_image(self, uris: List[str]) -> List[List[float]]:
        image_features = []
        for uri in uris:
            # 阿里dashscope SDK要求传递图片的地址，对于本地图片dashscope SDK会将图片上传到OSS服务中：
            local_image_uri = f"file://{uri}"
            resp = dashscope.MultiModalEmbedding.call(model=self.model, input=[{"image": local_image_uri}])
            while resp.output is None:
                print(f"{local_image_uri} 向量化失败！")
                resp = dashscope.MultiModalEmbedding.call(model=self.model, input=[{"image": local_image_uri}])

            embeddings_list = resp.output['embeddings'][0]['embedding']
            image_features.append(embeddings_list)
        return image_features

In [ ]:
# 1.6 嵌入图片与文本
# 1 向量数据库
vectorstore = Chroma(collection_name="multi-vector", embedding_function=MultiDashScopeEmbeddings())

# 2 获得图片的地址
image_uris = sorted(
    [
        os.path.join(IMAGE_OUT_DIR, image_name)
        for image_name in os.listdir(IMAGE_OUT_DIR)
        if image_name.endswith(".jpg")
    ]
)

# 3 添加图片 （存储图像base64数据与其向量数据）
print("向量化图片" + '-' * 100)
if image_uris:
    vectorstore.add_images(uris=image_uris)  # embedding_function 向量化
# 4 添加文本与表格
print("向量化表格" + '-' * 100)
if tables:
    vectorstore.add_documents([Document(page_content=table) for table in tables])  # embedding_function 向量化
print("向量化文本" + '-' * 100)
if texts:
    vectorstore.add_documents([Document(page_content=text) for text in texts])
print("数据添加完毕!!!")

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
# 1.7 向量召回测试
query = "手电筒的电路模型"
docs = retriever.invoke(query)
print("1. 向量召回结果数： " + '-' * 100)
print(len(docs))
print("jupyter中展示图片：")
for doc in docs:
    if is_base64(doc.page_content):
        show_plt_img(doc.page_content)

In [ ]:
# 1.8 RAG构建
# 1 缩放图片目录构建
if os.path.exists(RESIZE_IMAGE_DIR):
    shutil.rmtree(RESIZE_IMAGE_DIR)
os.makedirs(RESIZE_IMAGE_DIR)


# 2 提示构造函数
def prompt_func(data_dict):
    # "context":{"images": ["缩放后的图片地址","缩放后的图片地址"], "texts": "doc1"}
    # 提取图片与文本
    images = data_dict["context"]["images"]
    texts = data_dict["context"]["texts"]
    messages = []
    # 装载图片数据
    for image in images:
        # 样例：
        # HumanMessage(content=[{'text': '请将图片标记标注为：`C:\\...\\35589c8d-e802-4f7b-a647-41ed1a0a3439.jpg`'},
        # {'image': 'file://C:\\...\\35589c8d-e802-4f7b-a647-41ed1a0a3439.jpg'}],
        # additional_kwargs={},
        # response_metadata={}
        # )
        messages.append(
            HumanMessage(
                content=[
                    {"text": f"请将图片标记标注为：`{image}`"},
                    {"image": f"file://{image}"}
                ]
            )
        )

    # 装载文本数据
    formatted_texts = "\n\n".join(texts)
    messages.append(
        # 样例：
        # HumanMessage(content=[{'text': '你是作为一名专...理排版，答案中提及的图片统一以`<image>传递的图片真实标记</image>`的形式呈现。\n
        # 用户的问题是：手电筒的电路模型\n\n参考的文本或者表格数据：\n'}],
        # additional_kwargs={},
        # response_metadata={})]
        HumanMessage(content=[
            {
                "text":
                    "你是作为一名专业的电气工程师和电路理论专家。你的任务是用中文回答与电路基本概念和定律相关的问题。"  # 角色
                    "你将获得相关的图片与文本作为参考的上下文。这些图片和文本都是根据用户输入的关键词从向量数据库中检索获取的。"
                    "请根据提供的图片和文本结合你丰富的知识与分析能力，提供一份全面的问题解答。"  # 任务
                    "请将提供的图片标记自然融入答案阐述的对应位置进行合理排版，答案中提及的图片统一以`<image>传递的图片真实标记</image>`的形式呈现。\n\n"  # 需求
                    f"用户的问题是：{data_dict['question']}\n\n"
                    "参考的文本或者表格数据：\n"  # 样例
                    f"{formatted_texts}"
            }
        ]
        )
    )

    print('*' * 200)
    print(f"messages: {messages}")
    print('*' * 200)
    return messages


# 3. 构建提示链
chain = (
        {
            "question": RunnablePassthrough(),
            "context": retriever | RunnableLambda(split_image_text_types)
        } | RunnableLambda(prompt_func)
)
result = chain.invoke(query)
print("2. 提示结果：" + '-' * 100)
print(result)

# 4 构建RAG链
llm = ChatTongyi(model="qwen-vl-max")
chain = (
        {
            "question": RunnablePassthrough(),
            "context": retriever | RunnableLambda(split_image_text_types)
        }
        | RunnableLambda(prompt_func)
        | llm
        | StrOutputParser()
)

result = chain.invoke(query)
print('3. RAG结果:' + '-' * 100)
print(result)

print("jupyter中展示结果:" + '-' * 100)
display_answer(result)

### 2.基于多模态大模型（图像摘要 + 文本摘要 + 多模态生成）

In [ ]:
# 2.1 初始化
import base64
import os
import shutil
import uuid
from pathlib import Path

import markdown
from IPython.display import HTML, display
from PIL import Image
from langchain_chroma import Chroma
from langchain_classic.retrievers import MultiVectorRetriever
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.stores import InMemoryStore
from unstructured.documents.elements import Table, CompositeElement
from unstructured.partition.pdf import partition_pdf

# 环境配置
os.environ["TESSERACT_CMD"] = "/usr/bin/tesseract"

# 初始化
BASE_DIR = "/mnt/c/大模型/智泊大模型全栈教程总结/02-教材整理 L2/代码/Langchain/6.langchain高级RAG/data/"
RESOURCE_DIR = BASE_DIR + 'resources/'
TOOLS_DIR = BASE_DIR + 'tools/'
IMAGE_OUT_DIR = RESOURCE_DIR + 'images/'
PDF_PATH = RESOURCE_DIR + "978-7-5170-2271-8_1.pdf"
RESIZE_IMAGE_DIR = RESOURCE_DIR + "temp"


In [ ]:
# 2.2 功能函数
def is_image_path(filepath):
    ''' 判断是否为图片地址 '''
    try:
        path = Path(filepath)
        return all([
            path.exists(),
            path.is_file(),
            path.suffix.lower() == '.jpg'
        ])
    except Exception:
        return False


def resize_base64_image4tongyi(image_path, max_size=(640, 480)):
    try:
        # 打开图片
        img = Image.open(image_path)
        width, height = img.size
        ratio = min(max_size[0] / width, max_size[1] / height)
        # 计算按比例缩放后的宽高
        new_width = int(width * ratio)
        new_height = int(height * ratio)

        # 缩放
        resized_img = img.resize((new_width, new_height), Image.LANCZOS)
        # 保存图片到指定路径
        out_path = out_path = os.path.join(RESIZE_IMAGE_DIR, str(uuid.uuid4()) + ".jpg")
        resized_img.save(out_path)
        # 图片地址
        return out_path
    except Exception as e:
        print(f"处理图片时出错: {e}")
        return None


def split_image_text_types(docs):
    """
    拆分图像和文本
    """
    images = []
    texts = []
    # docs：文本内容和图片地址
    for doc in docs:
        if is_image_path(doc):
            doc = resize_base64_image4tongyi(doc)
            images.append(doc)
        else:
            texts.append(doc)
    return {"images": images, "texts": texts}


def show_plt_img(img_base64):
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    display(HTML(image_html))


def encode_image(img_path):
    with open(img_path, "rb") as img_file:
        img_data = img_file.read()
        img_base64 = base64.b64encode(img_data).decode('utf-8')
        return img_base64


def display_answer(text: str):
    start_tag = "<image>"
    end_tag = "</image>"
    parts = text.split(start_tag)
    for part in parts:
        # 再根据</image>标签 分割文本：xxx => ['xxx'], xxx</image> => ['xxx','']
        chunks = part.split(end_tag)
        if len(chunks) > 1:
            # 存在图片
            image_path = chunks[0]
            context = chunks[1]
            img_base64 = encode_image(image_path)
            display(HTML(f'\n<img src="data:image/jpeg;base64,{img_base64}"/>\n'))
            # display(HTML(context.replace("\n", "<br/>")))
            display(HTML(markdown.markdown(context)))
        else:
            display(HTML(markdown.markdown(part)))


In [ ]:
# 2.3 数据加载
# 如果图片提取目录存在则删除重建
if os.path.exists(IMAGE_OUT_DIR):
    shutil.rmtree(IMAGE_OUT_DIR)
os.makedirs(IMAGE_OUT_DIR)

print('\n开始解析pdf文档' + '-' * 100)
pdf_data = partition_pdf(
    filename=PDF_PATH,
    extract_images_in_pdf=True,  # 提取图片
    infer_table_structure=True,  # 启用表格结构识别
    max_characters=4000,  # 每个文本块最大字符数
    new_after_n_chars=3800,  # 达到3800字符后分新块
    combine_text_under_n_chars=2000,  # 合并小于2000字符的文本块
    chunking_strategy="by_title",  # 按标题分块
    extract_image_block_output_dir=IMAGE_OUT_DIR,  # 图片提取路径
)

In [ ]:
# 2.4 提取表格与文本
tables = []
texts = []
for element in pdf_data:
    if isinstance(element, Table):
        tables.append(str(element))
    elif isinstance(element, CompositeElement):
        texts.append(str(element))
print(f"表格元素：{len(tables)}  文本元素：{len(texts)}")

# 4. 生成文本和表格摘要
prompt = PromptTemplate.from_template(
    "你是一位负责生成表格和文本摘要以供检索的助理。"  # 角色
    "这些摘要将被嵌入并用于检索原始文本或表格元素。"
    "请提供表格或文本的简明摘要，该摘要已针对检索进行了优化。表格或文本：{document}"  # 任务
)

In [ ]:
# 2.5 使用大模型生成文本摘要
model = ChatTongyi(model="qwen-max")
summarize_chain = {"document": lambda x: x} | prompt | model | StrOutputParser()
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

# 生成图片摘要
def image_summarize(image_path):
    """生成图片摘要"""
    chat = ChatTongyi(model="qwen-vl-max")
    local_image_path = f"file://{image_path}"
    response = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"text": "你是一名负责生成图像摘要以便检索的助理。这些摘要将被嵌入并用于检索原始图像。"  # 角色
                             "请生成针对检索进行了优化的简洁的图像摘要。"},  # 任务
                    {"image": local_image_path}
                ]
            )
        ]
    )
    return response.content


# 检索图片摘要获得图片地址
img_list = []  # 原图片地址
image_summaries = []  # 图片摘要
for img_file in sorted(os.listdir(IMAGE_OUT_DIR)):
    if img_file.endswith(".jpg"):
        img_path = os.path.join(IMAGE_OUT_DIR, img_file)
        img_list.append(img_path)
        # 生成图片摘要
        image_summaries.append(image_summarize(img_path)[0]["text"])

In [ ]:
# 2.6 构建向量索引（摘要索引）
embeddings_model = DashScopeEmbeddings(
    model="text-embedding-v1",
)

# 创建向量数据库（用于存储摘要）
vectorstore = Chroma(
    collection_name="multi_model",
    embedding_function=embeddings_model
)

# 创建内存存储（用于存储原内容）
docstore = InMemoryStore()
# 将摘要存储入库
id_key = "doc_id"

def add_documents(doc_summaries, doc_contents):
    if not doc_summaries or not doc_contents:
        print("警告：文档摘要或内容为空，跳过添加操作。")
        return

    if len(doc_summaries) != len(doc_contents):
        print("警告：文档摘要和内容的数量不匹配，跳过添加操作。")
        return

    doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
    summary_docs = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(doc_summaries)
    ]
    vectorstore.add_documents(summary_docs)
    docstore.mset(list(zip(doc_ids, doc_contents)))


add_documents(text_summaries, texts)
add_documents(table_summaries, tables)
add_documents(image_summaries, img_list)

# 构建多向量检索（摘要索引）
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key=id_key,
    search_kwargs={"k": 7}
)

In [ ]:
# 2.7.RAG构建
# 构建缩放图片存放目录
if os.path.exists(RESIZE_IMAGE_DIR):
    shutil.rmtree(RESIZE_IMAGE_DIR)
os.makedirs(RESIZE_IMAGE_DIR)


# RAG构建
def prompt_func(data_dict):
    # "context":{"images": ["缩放后的图片地址","缩放后的图片地址"], "texts": "doc1"}
    # 提取图片与文本
    images = data_dict["context"]["images"]
    texts = data_dict["context"]["texts"]
    messages = []
    # 装载图片数据
    for image in images:
        messages.append(
            HumanMessage(
                content=[
                    {"text": f"请将图片标记标注为：`{image}`"},
                    {"image": f"file://{image}"}
                ]
            )
        )

    # 装载文本数据
    formatted_texts = "\n\n".join(texts)
    # 该提示词可优化，如加入少量样本，让大模型输出更稳定
    messages.append(
        HumanMessage(content=[
            {
                "text":
                    "你是作为一名专业的电气工程师和电路理论专家。你的任务是用中文回答与电路基本概念和定律相关的问题。"  # 角色
                    "你将获得相关的图片与文本作为参考的上下文。这些图片和文本都是根据用户输入的关键词从向量数据库中检索获取的。"
                    "请根据提供的图片和文本结合你丰富的知识与分析能力，提供一份全面的问题解答。"  # 任务
                    "请将提供的图片标记自然融入答案阐述的对应位置进行合理排版，答案中提及的图片统一以`<image>传递的图片真实标记</image>`的形式呈现。\n\n"  # 限制
                    f"用户的问题是：{data_dict['question']}\n\n"
                    "参考的文本或者表格数据：\n"
                    f"{formatted_texts}"
            }
        ]
        )
    )

    print('*' * 200)
    print(f"messages: {messages}")
    print('*' * 200)
    return messages


# 千问视觉模型（多模态）
llm = ChatTongyi(model="qwen-vl-max")
chain = (
        {
            "question": RunnablePassthrough(),
            "context": retriever | RunnableLambda(split_image_text_types)
        }
        | RunnableLambda(prompt_func)
        | llm
        | StrOutputParser()
)

In [ ]:
# 2.8 效果展示
# 向量召回展示
query = "介绍下手电筒的电路模型"
docs = retriever.invoke(query)

print("1. 向量召回结果展示在jupyter中" + '-' * 100)
for doc in docs:
    if is_image_path(doc):
        show_plt_img(encode_image(doc))

# RAG召回展示
result = chain.invoke(query)
print("2. RAG召回结果" + '-' * 100)
display_answer(result)


### 3.基于多模态大模型（图像摘要 + 文本向量检索 + 文本生成）

In [ ]:
# 3.1 初始化
import base64
import os
import re
import shutil
import uuid

from IPython.display import HTML, display, Markdown
from langchain_chroma import Chroma
from langchain_classic.retrievers import MultiVectorRetriever
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.stores import InMemoryStore
from unstructured.documents.elements import Image as ImageElement
from unstructured.partition.pdf import partition_pdf

# 环境配置
os.environ["TESSERACT_CMD"] = "/usr/bin/tesseract"

# 1. 初始化
BASE_DIR = "/mnt/c/大模型/智泊大模型全栈教程总结/02-教材整理 L2/代码/Langchain/6.langchain高级RAG/data/"
RESOURCE_DIR = BASE_DIR + 'resources/'
TOOLS_DIR = BASE_DIR + 'tools/'
IMAGE_OUT_DIR = RESOURCE_DIR + 'images/'
PDF_PATH = RESOURCE_DIR + "978-7-5170-2271-8_1.pdf"
RESIZE_IMAGE_DIR = RESOURCE_DIR + "temp"

In [ ]:
# 2. 加载pdf
# 如果图片提取目录存在则删除重建
if os.path.exists(IMAGE_OUT_DIR):
    shutil.rmtree(IMAGE_OUT_DIR)
os.makedirs(IMAGE_OUT_DIR)

# 使用unstructured库解析PDF文档(需要科学上网)
pdf_data = partition_pdf(
    filename=PDF_PATH,
    extract_images_in_pdf=True,
    infer_table_structure=True,  # 启用表格结构识别
    max_characters=4000,  # 每个文本块最大字符数
    new_after_n_chars=3800,  # 达到3800字符后分新块
    combine_text_under_n_chars=2000,  # 合并小于2000字符的文本块
    chunking_strategy="by_title",  # 按标题分块
    extract_image_block_output_dir=IMAGE_OUT_DIR,  # 图片提取路径
)

In [ ]:
# 3. 生成摘要  生成文本和表格摘要
prompt = PromptTemplate.from_template(
    "你是一位负责生成表格和文本摘要以供检索的助理。"  # 角色
    "这些摘要将被嵌入并用于检索原始文本或表格元素。"
    "请提供表格或文本的简明摘要，该摘要已针对检索进行了优化。表格或文本：{document}"  # 任务
)

# 使用大模型生成文本摘要
model = ChatTongyi(model="qwen-max")
summarize_chain = {"document": lambda x: x.text} | prompt | model | StrOutputParser()
summaries = summarize_chain.batch(pdf_data, {"max_concurrency": 5})


def image_summarize(image_path):
    """生成图片摘要"""
    chat = ChatTongyi(model="qwen-vl-max")
    local_image_path = f"file://{image_path}"
    print(f"生成摘要:{image_path}")
    response = chat.invoke(
        [
            HumanMessage(
                content=[
                    {
                        "text": "你是一名负责生成图像摘要以便检索的助理。这些摘要将被嵌入并用于检索原始图像。请生成针对检索进行了优化的简洁的图像摘要。"},
                    {"image": local_image_path}
                ]
            )
        ]
    )
    return response.content


image_summaries = []
# image_list = []
for element in pdf_data:
    orig_elements = element.metadata.orig_elements
    length = len(orig_elements)
    for i, orig_element in enumerate(orig_elements):
        # 图片元素
        if isinstance(orig_element, ImageElement):
            image_path = orig_element.metadata.to_dict()["image_path"]
            # image_list.append(orig_element)
            # 将图片摘要记录在图片元素的text属性中
            summarizes = image_summarize(image_path)[0]["text"]
            orig_element.text = summarizes
            image_summaries.append(summarizes)

In [ ]:

# 4. 构建索引
embeddings_model = DashScopeEmbeddings(model="text-embedding-v1")
# 创建向量数据库（用于存储摘要）
vectorstore = Chroma(
    collection_name="multi_model_opt",
    embedding_function=embeddings_model
)
# 创建内存存储（用于存储原内容）
docstore = InMemoryStore()
# 将摘要存储入库
id_key = "doc_id"


def add_documents(doc_summaries, doc_contents):
    doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
    summary_docs = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(doc_summaries)
    ]
    vectorstore.add_documents(summary_docs)
    docstore.mset(list(zip(doc_ids, doc_contents)))


# 不再单独放入图片摘要??
add_documents(summaries, pdf_data)  # PDF Element

In [ ]:

# 5. 构建多向量检索（摘要索引）
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key=id_key,
    search_kwargs={"k": 2}
)


In [ ]:
# 6. RAG构建
def split_image_text_types(docs):
    texts = []
    # 注意：doc为PDF Element
    for doc in docs:
        text = ''
        # 从文档元数据中获取原始元素列表，这些元素可能包含文本、图片等不同类型
        orig_element = doc.metadata.orig_elements
        for element in orig_element:
            # 是否为图片元素
            if isinstance(element, ImageElement):
                # 图片元素用标记包裹  src=图片路径  element.text=图片相关文本  例如： <image src="/path/to/image.png">图片描述文字</image>
                text += f'<image src="{element.metadata.image_path}">{element.text}</image>'
            else:
                # 其他元素直接放入文本
                text += element.text
        texts.append(text)
    return texts


def prompt_func(data_dict):
    question = data_dict["question"]
    context = data_dict["context"]
    # 装载数据
    formatted_texts = "\n\n".join(context)

    prompt = ("你是作为一名专业的电气工程师和电路理论专家。你的任务是用中文回答与电路基本概念和定律相关的问题。"
              "你将获得相关文档作为参考的上下文。这些文档都是根据用户输入的关键词从向量数据库中检索获取的。"
              "请根据提供的文档结合你丰富的知识与分析能力，提供一份全面的问题解答。"
              r"请返回Markdown格式数据，并且当涉及到数学公式时，请使用正确的LaTeX语法来编写这些公式，对于行内公式应该以单个美元符号`$`;对于独立成行的公式，使用双美元符号`$$`包裹。例如，行内公式：`$a = \frac{1}{2}$`,而独立成行的公式则是：`$$ a = \frac{1}{2} $$`。"
              "请将提供的文档中的图片`<image src='...'>`(不包括`<image>`)到`</image>`中间的文字，在需要时自然融入答案阐述的对应位置进行合理排版。\n\n"
              f"用户的问题是：\n{question}\n\n"
              "参考的文本或者表格数据：\n"
              f"{formatted_texts}")

    print('*' * 200)
    print(f"prompt: {prompt}")
    print('*' * 200)

    return prompt


llm = ChatTongyi(model="qwen-max")
chain = (
        {
            "question": RunnablePassthrough(),
            "context": retriever | RunnableLambda(split_image_text_types)
        }
        | RunnableLambda(prompt_func)
        | llm
        | StrOutputParser()
)

In [ ]:

# 7. 效果展示
result = chain.invoke("介绍下电路模型")


def encode_image(img_path):
    with open(img_path, "rb") as img_file:
        img_data = img_file.read()
        img_base64 = base64.b64encode(img_data).decode('utf-8')
        return img_base64


def display_answer(text: str):
    # 正则表达式：用于匹配 <image src="xxx"> 并捕获 src 属性的值
    pattern = r'(<image src="([^"]*)">)'
    # abc<image src="xxx">def -》 ["abc", "<image src="xxx">", "xxx", "def"]
    chunks = re.split(pattern, text)
    for i, chunk in enumerate(chunks):
        # 文本内容
        if i % 3 == 0:
            display(Markdown(chunk.replace("</image>", "")))
        elif i % 3 == 2:
            # image_path = re.search(r'src="([^"]*)"', chunk).group(1) 1% 3==1
            img_base64 = encode_image(chunk)
            display(HTML(f'\n<img src="data:image/jpeg;base64,{img_base64}"/>\n'))


print('RAG结果展示:' + '-' * 100)
display_answer(result)
